In [ ]:
!pip install -U transformers
!pip install datasets --upgrade --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6

In [ ]:
import transformers
print(transformers.__version__)
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling


4.53.0


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import torch
# ← 여기 YOUR_HF_TOKEN 부분에 본인의 hf_… 토큰을 붙여넣으세요!
os.environ["HF_TOKEN"] = "toekn_here"

from huggingface_hub import login
login(token=os.environ["HF_TOKEN"])
model_id ="meta-llama/Llama-3.1-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
      model_id,
      device_map="auto",
      torch_dtype=torch.bfloat16
)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]



---



---
# 데이터 전처리

---



---



In [ ]:
# ============================================================
# Parquet 파일 직접 다운로드
# ============================================================
from huggingface_hub import hf_hub_download
import pathlib

DATA_DIR = "/content/light_dialog_parquet"
pathlib.Path(DATA_DIR).mkdir(exist_ok=True, parents=True)

train_file = hf_hub_download(
    repo_id="dap-exp/light_dialog",
    repo_type="dataset",
    filename="data/train-00000-of-00001.parquet",
    local_dir=DATA_DIR
)
valid_file = hf_hub_download(
    repo_id="dap-exp/light_dialog",
    repo_type="dataset",
    filename="data/valid-00000-of-00001.parquet",
    local_dir=DATA_DIR
)

print("✅ 다운로드된 파일:")
print("  train:", train_file)
print("  valid:", valid_file)

# ============================================================
# pandas 로 Parquet → Python 리스트 변환
# ============================================================
import pandas as pd

df_train = pd.read_parquet(train_file)
df_valid = pd.read_parquet(valid_file)

train_raw = df_train.to_dict(orient="records")
valid_raw = df_valid.to_dict(orient="records")

print("원본 개수 (train/valid):", len(train_raw), len(valid_raw))
print("예시 키:", train_raw[0].keys())

# ============================================================
# Llama-3.1 토크나이저 로드
# ============================================================
from transformers import AutoTokenizer

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    use_fast=True,
    use_auth_token=True
)
tokenizer.pad_token = tokenizer.eos_token

# ============================================================
# (prompt, response) 쌍 생성 함수 정의
# ============================================================
MAX_CTX = 4  # 프롬프트에 포함할 직전 대화 턴 수

def make_pairs(example):
    dlg   = example["dialogue"]
    chars = example["characters"]
    self_nm    = chars["self_name"]
    partner_nm = chars["partner_name"]
    persona    = chars.get("self_persona", "")
    setting    = example["setting"]["description"]

    # 시스템 프롬프트: 역할·배경·페르소나
    sys_hdr = (
        "# system\n"
        f'You are "{self_nm}".\n'
        f"Setting: {setting}\n"
        f"Persona: {persona}\n"
    )

    out = []
    for i in range(len(dlg) - 1):
        start = max(0, i - MAX_CTX + 1)
        ctx = dlg[start : i + 1]

        # "화자: 발화" 형태로 문맥 조합
        lines = []
        for j, utt in enumerate(ctx):
            idx = start + j
            speaker = partner_nm if idx % 2 == 0 else self_nm
            lines.append(f"{speaker}: {utt}")

        prompt = sys_hdr + "# user\n" + "\n".join(lines) + "\n# assistant\n"
        response = dlg[i + 1]
        out.append({"prompt": prompt, "response": response})
    return out

def build_pair_list(raw_list):
    all_pairs = []
    for ex in raw_list:
        all_pairs.extend(make_pairs(ex))
    return all_pairs

train_pairs = build_pair_list(train_raw[:4000])
valid_pairs = build_pair_list(valid_raw[:2500])

print("생성된 쌍 개수 (train/valid):", len(train_pairs), len(valid_pairs))
print("prompt 예시:\n", train_pairs[0]["prompt"])

# ============================================================
# 토크나이즈 + labels 생성
# ============================================================
from datasets import Dataset

MAX_LEN = 512  # 토큰 최대 길이

def tokenize_batch(batch):
    enc_p = tokenizer(
        batch["prompt"],
        padding="max_length",
        truncation=True,
        max_length=MAX_LEN
    )
    enc_r = tokenizer(
        batch["response"],
        padding="max_length",
        truncation=True,
        max_length=MAX_LEN
    )
    enc_p["labels"] = enc_r["input_ids"]
    return enc_p

# train_ds 생성
train_ds = Dataset.from_list(train_pairs)
train_ds = train_ds.map(
    tokenize_batch,
    batched=True,
    remove_columns=["prompt", "response"]
)
train_ds.set_format(type="torch")

# valid_ds 생성
valid_ds = Dataset.from_list(valid_pairs)
valid_ds = valid_ds.map(
    tokenize_batch,
    batched=True,
    remove_columns=["prompt", "response"]
)
valid_ds.set_format(type="torch")

print("Train columns:", train_ds.column_names)
print("Valid columns:", valid_ds.column_names)

# ============================================================
# 디스크에 저장 (학습 시 load_from_disk 로 불러오기)
# ============================================================
train_ds.save_to_disk("/content/light_dialog_train_llama31")
valid_ds.save_to_disk("/content/light_dialog_valid_llama31")

print("✅ 전처리 완료! 저장 경로:")
print("  /content/light_dialog_train_llama31")
print("  /content/light_dialog_valid_llama31")


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


train-00000-of-00001.parquet:   0%|          | 0.00/8.12M [00:00<?, ?B/s]

valid-00000-of-00001.parquet:   0%|          | 0.00/481k [00:00<?, ?B/s]

✅ 다운로드된 파일:
  train: /content/light_dialog_parquet/data/train-00000-of-00001.parquet
  valid: /content/light_dialog_parquet/data/valid-00000-of-00001.parquet
원본 개수 (train/valid): 17075 1000
예시 키: dict_keys(['task', 'setting', 'characters', 'dialogue'])


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:934: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


...

Map:   0%|          | 0/10268 [00:00<?, ? examples/s]

Train columns: ['input_ids', 'attention_mask', 'labels']
Valid columns: ['input_ids', 'attention_mask', 'labels']


Saving the dataset (0/2 shards):   0%|          | 0/81357 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10268 [00:00<?, ? examples/s]

✅ 전처리 완료! 저장 경로:
  /content/light_dialog_train_llama31
  /content/light_dialog_valid_llama31




---



---

# LoRA

---



---



In [ ]:
from peft import LoraConfig,get_peft_model,TaskType

lora_config = LoraConfig (
    r = 8, # 저랭크 차원 수 클수록 표현련 , 파라미터수 , 연산량 증가
    lora_alpha = 16, # scaling factor
    lora_dropout = 0.1,
    bias ="none", #기존 레이어의 bias 파라미터 처리 방식
    task_type=TaskType.CAUSAL_LM, # 모델 작업 유형 LLaMA류
    target_modules=["q_proj", "v_proj"] # LoRA를 적용할 모델의 모듈 이름 목
)

model = get_peft_model(model,lora_config)
model.to("cuda")

model.print_trainable_parameters() # 몇개만 학습되는지 확인하는 용도

trainable params: 3,407,872 || all params: 8,033,669,120 || trainable%: 0.0424




---



---

# Trainer


---



---



In [ ]:
import torch
torch.cuda.empty_cache()

In [8]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)
from datasets import load_from_disk

train_dataset = load_from_disk("/content/light_dialog_train_llama31")
eval_dataset = load_from_disk("/content/light_dialog_valid_llama31")

tokenizer.pad_token = tokenizer.eos_token  # 또는 아래처럼 따로 추가해도 됨
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})
import torch
torch.cuda.empty_cache()

training_args = TrainingArguments(
    output_dir="./lora-LLaMa",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    bf16=True,
    learning_rate=2e-4,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer = tokenizer,
    mlm = False
)

trainer = Trainer (
    model = model,
    args=training_args,
    train_dataset =train_dataset,
    eval_dataset = eval_dataset,
    tokenizer = tokenizer,
    data_collator = data_collator,
)
trainer.train()

best_model_dir="./best_model"
trainer.model.save_pretrained(best_model_dir)
tokenizer.save_pretrained(best_model_dir)



/tmp/ipython-input-8-3227967205.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer (
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,1.063400,1.147537
2,1.020000,1.129283


Epoch,Training Loss,Validation Loss
1,1.063400,1.147537
2,1.020000,1.129283
3,0.908900,1.136791


('/content/drive/MyDrive/best_model/tokenizer_config.json',
 '/content/drive/MyDrive/best_model/special_tokens_map.json',
 '/content/drive/MyDrive/best_model/chat_template.jinja',
 '/content/drive/MyDrive/best_model/tokenizer.json')